In [1]:
import json
import requests
from unittest import TestCase

case = TestCase()

In [2]:
LOCAL = 'http://localhost:8000/'

In [3]:
with open('../tests/testes.json', 'r') as json_in:
    testes = json.load(json_in)
print(testes)

{'DTSC': {'imonavio': 'string', 'cargas': [{'codigorecinto': 'string', 'cpfcnpjproprietario': 'string', 'placa': 'string', 'numero': 'string', 'tipodocumentotransporte': 'CE', 'documentotransporte': 'string', 'cpfcnpjtransportador': 'string', 'placasemireboque': 'string'}], 'operadorregistro': 'string', 'dataregistro': '2019-05-28T11:01:40.071Z', 'dataoperacao': '2019-05-28T11:01:40.071Z', 'dataevento': '2019-05-28T11:01:40.071Z', 'operadorevento': 'string', 'retificador': False, 'IDEvento': 0, 'viagem': 'string'}, 'PosicaoLote': {'posicao': 'string', 'operadorregistro': 'string', 'dataregistro': '2019-05-28T11:16:53.793Z', 'numerolote': 0, 'dataevento': '2019-05-28T11:16:53.793Z', 'operadorevento': 'string', 'retificador': False, 'IDEvento': 0, 'qtdevolumes': 'string'}, 'Desunitizacao': {'retificador': False, 'placasemireboque': 'string', 'numero': 'string', 'tipodocumentotransporte': 'CE', 'documentotransporte': 'string', 'operadorevento': 'string', 'lotes': [{'marca': 'string', 'fal

In [4]:
for classe, teste in testes.items():
    rv = requests.post(LOCAL + classe.lower(), json=teste)
    print(classe)
    print(rv.status_code)
    print(rv.text)

DTSC
201
{
  "detail": -4889031623810489004,
  "status": 201,
  "title": "Evento incluido"
}

PosicaoLote
201
{
  "detail": 4255169367675889654,
  "status": 201,
  "title": "Evento incluido"
}

Desunitizacao
201
{
  "detail": 5306747965044602688,
  "status": 201,
  "title": "Evento incluido"
}

ArtefatoRecinto
201
{
  "detail": 12229789312474444,
  "status": 201,
  "title": "Evento incluido"
}

OperacaoNavio
201
{
  "detail": -5722745027502104445,
  "status": 201,
  "title": "Evento incluido"
}

AcessoPessoa
201
{
  "detail": -424119137747152305,
  "status": 201,
  "title": "Evento incluido"
}

PesagemVeiculoVazio
201
{
  "detail": -249668476941131943,
  "status": 201,
  "title": "Evento incluido"
}

InspecaonaoInvasiva
201
{
  "detail": -6994470354546900517,
  "status": 201,
  "title": "Evento incluido"
}

AcessoVeiculo
201
{
  "detail": -6687942510608380794,
  "status": 201,
  "title": "Evento incluido"
}

PosicaoConteiner
201
{
  "detail": 1413249831738137937,
  "status": 201,
  "ti

In [5]:
def extractDictAFromB(A, B):
    return dict([(k, B[k]) for k in A.keys() if k in B.keys()])


In [6]:
ltestes = testes.copy()
for classe, teste in ltestes.items():
    print(classe)
    rv = requests.get(LOCAL + classe.lower() + '/' + str(teste['IDEvento']))
    print(rv.status_code)
    if rv.status_code == 200:
        response_json = rv.json()
        for data in ['dataevento', 'dataregistro', 'dataoperacao', 'dataliberacao']:
            if teste.get(data) is not None:
                teste.pop(data)
            if response_json.get(data) is not None:
                response_json.pop(data)
        try:
            sub_response = extractDictAFromB(teste, response_json)
            # case.assertDictEqual(teste, sub_response)
            '''
            for key, value in teste.items():
                if isinstance(value, list):
                    for item in value:
                        response_item =  sub_response.get(key)
                        if response_item is not None:
                            print(value)
                            print(response_item)
                            # case.assertEqual(value, response_item)
            '''
            case.assertDictContainsSubset(teste, sub_response)
            # case.assertDictEqual(lteste, response_json)
        except AssertionError as err:
            print(err)
    # print(rv.text)

PosicaoVeiculo
200
Unitizacao
200
InspecaonaoInvasiva
200
Mismatched values: 'anexos', expected: [{'content': '', 'contentType': '', 'datamodificacao': '2019-05-28T12:18:41.204Z', 'nomearquivo': '', 'datacriacao': '2019-05-28T12:18:41.204Z'}], actual: [{'content': '', 'contentType': '', 'datamodificacao': '2019-05-28T12:18:41.204000Z', 'nomearquivo': '', 'datacriacao': '2019-05-28T12:18:41.204000Z'}]
DTSC
200
PosicaoLote
200
Desunitizacao
200
ArtefatoRecinto
200
PesagemTerrestre
200
PesagemVeiculoVazio
200
OperacaoNavio
200
AcessoVeiculo
200
Mismatched values: 'dataagendamento', expected: '2019-05-31T12:10:45.083Z', actual: None
PosicaoConteiner
200
Ocorrencia
200
PesagemMaritimo
200
AcessoPessoa
200
AvariaLote
200


In [7]:
import time
s0 = time.time()
query = {'IDEvento': -1,
       'tipoevento': 'PesagemMaritimo'}
r = requests.get(LOCAL + 'get_eventos_novos',
                  data=query)
s1 = time.time()
print(r.status_code)
print(r.text[:1000])

200
[
  {
    "ID": 1, 
    "IDEvento": 0, 
    "capturaautomatica": true, 
    "dataevento": "2019-05-28T11:45:43.499000Z", 
    "dataregistro": "2019-05-28T11:45:43.499000Z", 
    "documentotransporte": "string", 
    "hash": 511425862082860040, 
    "numero": "string", 
    "operadorevento": "string", 
    "operadorregistro": "string", 
    "pesobalanca": 0, 
    "pesobrutodeclarado": 0, 
    "placa": "string", 
    "placasemireboque": "string", 
    "recinto": "00001", 
    "request_IP": "127.0.0.1", 
    "retificador": false, 
    "taraconjunto": 0, 
    "time_created": "2019-06-05T13:25:15.339308Z", 
    "tipodocumentotransporte": "CE"
  }
]

